In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier
import math
import operator
import random
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

In [5]:
data_spam = pd.read_csv("C:/Users/mouni/Downloads/SML/HW4/spambase.data")

In [6]:
data_spam = data_spam.rename(columns={'0':'word_freq_make','0.64':'word_freq_address',
                                      '0.64.1':'word_freq_all','0.1':'word_freq_3d',
                                      '0.32':'word_freq_our','0.2':'word_freq_over',
                                      '0.3':'word_freq_remove','0.4':'word_freq_internet',
                                      '0.5':'word_freq_order','0.6':'word_freq_mail',
                                      '0.7':'word_freq_receive','0.64.2':'word_freq_will','0.8':'word_freq_people',
                                      '0.9':'word_freq_report','0.10':'word_freq_addresses','0.32.1':'word_freq_free',
                                      '0.11':'word_freq_business','1.29':'word_freq_email',
                                      '1.93':'word_freq_you','0.12':'word_freq_credit','0.96':'word_freq_your',
                                      '0.13':'word_freq_font','0.14':'word_freq_000','0.15':'word_freq_money',
                                      '0.16':'word_freq_hp','0.17':'word_freq_hpl','0.18':'word_freq_george',
                                      '0.19':'word_freq_650','0.20':'word_freq_lab','0.21':'word_freq_labs',
                                      '0.22':'word_freq_telnet','0.23':'word_freq_857','0.24':'word_freq_data',
                                      '0.25':'word_freq_415','0.26':'word_freq_85','0.27':'word_freq_technology',
                                      '0.28':'word_freq_1999','0.29':'word_freq_parts','0.30':'word_freq_pm',
                                      '0.31':'word_freq_direct','0.32.2':'word_freq_cs','0.33':'word_freq_meeting',
                                      '0.34':'word_freq_original','0.35':'word_freq_project','0.36':'word_freq_re',
                                      '0.37':'word_freq_edu','0.38':'word_freq_table','0.39':'word_freq_conference',
                                      '0.40':'char_freq_semicolon','0.41':'char_freq_leftparen',
                                      '0.42':'char_freq_leftsquare','0.778':'char_freq_bang',
                                      '0.43':'char_freq_dollar','0.44':'char_freq_hash',
                                      '3.756':'capital_run_length_average','61':'capital_run_length_longest',
                                      '278':'capital_run_length_total','1':'is_spam'})

In [7]:
data_spam.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_semicolon,char_freq_leftparen,char_freq_leftsquare,char_freq_bang,char_freq_dollar,char_freq_hash,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
0,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
1,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
2,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,1.85,0.00,0.00,1.85,0.00,0.00,...,0.00,0.223,0.0,0.000,0.000,0.000,3.000,15,54,1


In [8]:
data_y = data_spam['is_spam']
data_x = data_spam.drop(columns = ['is_spam'])
data_x.shape

(4600, 57)

In [9]:
scaler = StandardScaler() 
data_scaled = scaler.fit_transform(data_x)
data_scaled.shape

(4600, 57)

In [10]:
df = pd.DataFrame(data=data_scaled[:,0:],
                  index=data_scaled[:,0],
                  columns=['word_freq_make', 'word_freq_address','word_freq_all', 'word_freq_3d', 'word_freq_our',
                           'word_freq_over', 'word_freq_remove', 'word_freq_internet','word_freq_order', 'word_freq_mail',
                           'word_freq_receive','word_freq_will', 'word_freq_people', 'word_freq_report',
                           'word_freq_addresses', 'word_freq_free', 'word_freq_business','word_freq_email',
                           'word_freq_you', 'word_freq_credit', 'word_freq_your','word_freq_font', 'word_freq_000',
                           'word_freq_money', 'word_freq_hp','word_freq_hpl', 'word_freq_george', 'word_freq_650',
                           'word_freq_lab','word_freq_labs', 'word_freq_telnet', 'word_freq_857', 'word_freq_data',
                           'word_freq_415', 'word_freq_85', 'word_freq_technology', 'word_freq_1999','word_freq_parts',
                           'word_freq_pm', 'word_freq_direct', 'word_freq_cs','word_freq_meeting', 'word_freq_original',
                           'word_freq_project', 'word_freq_re', 'word_freq_edu', 'word_freq_table',
                           'word_freq_conference', 'char_freq_semicolon', 'char_freq_leftparen', 'char_freq_leftsquare',
                           'char_freq_bang', 'char_freq_dollar', 'char_freq_hash','capital_run_length_average',
                           'capital_run_length_longest', 'capital_run_length_total'])

In [11]:
df.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_semicolon,char_freq_leftparen,char_freq_leftsquare,char_freq_bang,char_freq_dollar,char_freq_hash,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0.345252,0.345252,0.051976,0.435261,-0.046905,-0.256087,0.672259,0.244655,-0.088058,-0.323341,1.086529,...,-0.111559,-0.158471,-0.026117,-0.155215,0.126330,0.423674,0.008739,-0.002453,0.250546,1.228189
-0.145982,-0.145982,-0.164984,0.851833,-0.046905,1.364700,0.343576,0.193562,0.036609,1.973754,0.016339,...,-0.111559,-0.117398,0.014571,-0.155215,0.008631,0.439942,-0.079768,0.145895,2.220875,3.258376
-0.342475,-0.342475,-0.164984,-0.556576,-0.046905,0.472524,-0.350309,0.500124,1.308212,0.789315,0.605719,...,-0.111559,-0.158471,-0.007622,-0.155215,-0.161788,-0.308392,-0.103060,-0.052154,-0.062450,-0.152207
-0.342475,-0.342475,-0.164984,-0.556576,-0.046905,0.472524,-0.350309,0.500124,1.308212,0.789315,0.605719,...,-0.111559,-0.158471,-0.015020,-0.155215,-0.164240,-0.308392,-0.103060,-0.052154,-0.062450,-0.152207
-0.342475,-0.342475,-0.164984,-0.556576,-0.046905,2.286616,-0.350309,-0.291828,4.350087,-0.323341,-0.371410,...,-0.111559,-0.158471,0.310487,-0.155215,-0.329755,-0.308392,-0.103060,-0.069079,-0.190726,-0.378150
